# Galileo OS FEC2 Reed-Solmon

This checks the test vector for the Galileo OS FEC2 Reed-Solomon coding using Phil Karn's libfec.
The encoding is defined in Appendix F of the [Galileo OS SIS ICD v2.0](https://www.gsc-europa.eu/sites/default/files/sites/all/files/Galileo_OS_SIS_ICD_v2.0.pdf). The test vectors are embedded as CSV files in that document.

The Reed-Solomon code is a shortened (118, 58) code based on the CCSDS code (in the conventional basis). The peculiarity of this code is that the order of the data bytes and of the parity check bytes are reversed (see below).

In [1]:
import numpy as np
import ctypes

In [2]:
information = np.fromfile('/tmp/Annex_F-RS_information_vector_example.csv', sep=';', dtype='uint8')
information

array([147, 109,  66,  23, 234, 140,  74, 234,  49,  89, 241, 253, 169,
       161,  89,  93,  75, 142,  83, 102,  98, 218,  14, 197, 155, 151,
        43, 181,   9, 163, 142, 111,   8, 118,  21,  47, 135, 139, 108,
       215,  51, 147, 185,  52,  17, 151,  97, 102, 238,  71,  83, 114,
        47,  80,  67, 199, 215, 162], dtype=uint8)

In [3]:
encoded = np.fromfile('/tmp/Annex_F-RS_code_vector_example.csv', sep=';', dtype='uint8')
encoded

array([147, 109,  66,  23, 234, 140,  74, 234,  49,  89, 241, 253, 169,
       161,  89,  93,  75, 142,  83, 102,  98, 218,  14, 197, 155, 151,
        43, 181,   9, 163, 142, 111,   8, 118,  21,  47, 135, 139, 108,
       215,  51, 147, 185,  52,  17, 151,  97, 102, 238,  71,  83, 114,
        47,  80,  67, 199, 215, 162, 238,  77,  12,  72, 235,  21, 148,
       213, 230,  54, 183,  82,  49, 104,  12, 228, 150, 157, 220, 112,
       236, 187,  63,  31, 175,  47, 210, 164,  17, 104,  98,  46, 252,
       165, 194,  57,  26, 213,  14, 133, 176, 148,  34,   9, 167,  43,
       204, 198,  25, 164, 233,  55, 153,  31, 237,  84, 212,  76, 137,
       242], dtype=uint8)

In [4]:
libfec = ctypes.CDLL('libfec.so')
libfec.init_rs_char.restype = ctypes.c_void_p
libfec.init_rs_char.argtypes = [ctypes.c_int] * 6
libfec.encode_rs_char.restype = None
libfec.encode_rs_char.argtypes = [
    ctypes.c_void_p, ctypes.POINTER(ctypes.c_char), ctypes.POINTER(ctypes.c_char)]

rs = libfec.init_rs_char(8, 0x11d, 1, 1, encoded.size - information.size, 255 - encoded.size)

output_buff = np.zeros(encoded.size - information.size, dtype='uint8')

In [5]:
# Invert the order of the data bytes before feeding them to the encoder.
input_data = information[::-1].copy()

libfec.encode_rs_char(
    rs,
    input_data.ctypes.data_as(ctypes.POINTER(ctypes.c_char)),
    output_buff.ctypes.data_as(ctypes.POINTER(ctypes.c_char)))

In [6]:
# Check that the encoder output matches the test vector. Note that we invert
# the order of the parity bytes produced by the encoder.
assert np.all(np.concatenate((information, output_buff[::-1])) == encoded)